In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import cftime
import xarray as xr
import wget
import os
import glob
import matplotlib.pyplot as plt
import dask

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

plt.gcf().autofmt_xdate()

<Figure size 2400x1600 with 0 Axes>

Les matrices SCRIBE sont le produit d'Environnement Canada que l'on veut répliquer à partir des données binaires du Datamart. Le traitement des données GRIB peut se faire en deux étapes, soit celle de la concaténation et celles du calcul au point de grille. Faisons donc ces manipulations avec les fichiers de l'émission 00z du 22 février 2021 (20210222) et pour la variable de la température moyenne (TMP)

In [2]:
grib_list=glob.glob('*.grib2')
grib_list

['CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P012.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P009.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P015.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P024.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P000.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P006.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P003.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P018.grib2',
 'CMC_glb_TMP_TGL_2_latlon.15x.15_2021022200_P021.grib2']

In [3]:
ds=xr.open_mfdataset(grib_list,concat_dim='valid_time',engine='cfgrib',combine='nested')

In [4]:
ds

<xarray.Dataset>
Dimensions:            (latitude: 1201, longitude: 2400, valid_time: 9)
Coordinates:
    time               datetime64[ns] 2021-02-22
    step               (valid_time) timedelta64[ns] 0 days 12:00:00 ... 21:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 -90.0 -89.85 -89.7 ... 89.7 89.85 90.0
  * longitude          (longitude) float64 -180.0 -179.8 -179.7 ... 179.7 179.9
  * valid_time         (valid_time) datetime64[ns] 2021-02-22T12:00:00 ... 20...
Data variables:
    t2m                (valid_time, latitude, longitude) float32 dask.array<chunksize=(1, 1201, 2400), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-02-26T13:30:11 GRIB to CDM+CF via cfgrib-0....

In [5]:
# Convert to celsius
ds=ds-273.15 #passage en celsius
t2m = ds.t2m 

# copy attributes to get nice figure labels and change Kelvin to Celsius
t2m.attrs = ds.t2m.attrs

t2m.attrs["units"] = "deg C"

In [8]:
t2m.isel(valid_time=1).plot()

Les fichiers grib d'environnement canada couvrent tout la planète. Je vais donc procèder à circonscrire exclusivement la zone du Québec.

In [9]:
min_lon=-45
min_lat=50
max_lon=-55
max_lat=60

mask_lon= (ds.longitude >=min_lon) & (ds.longitude >=max_lon)
mask_lat= (ds.latitude >=min_lat) & (ds.latitude >=max_lat)

cropped_ds = ds.where(mask_lon & mask_lat, drop=True)
cropped_ds

In [10]:
cropped_ds.t2m.isel(valid_time=1).plot()

Ajout de la carte du canada ainsi que des lignes de transport dans la carte

In [12]:
#admettons pour la station 
t2m1d = t2m.isel(latitude=10, longitude=10)
t2m1d.plot()